# eFEL settings and how to use them

This notebook will show you how to use the settings associated to the eFEL library that BluePyEfe use to compute the electric features (efeatures). These settings can allow for example to extract efeatures on a specific sections of a voltage recording, or to change the voltage threshold at which action potential are detected as such.

For this example we will use the rat somatosensory cortex data available at: http://microcircuits.epfl.ch/#/article/article_4_eph

Author of this script: Tanguy Damart @ Blue Brain Project

## How to specify eFEL settings

Let's start by creating a Cell and associating it a recording (see notebook example_of_extraction for a tutorial):

In [11]:
from bluepyefe.cell import Cell

cell = Cell(name="MouseNeuron")

files_metadata = {
    "i_file": "../tests/exp_data/B95_Ch0_IDRest_107.ibw",
    "v_file": "../tests/exp_data/B95_Ch3_IDRest_107.ibw",
    "i_unit": "pA",
    "v_unit": "mV",
    "t_unit": "s",
    "dt": 0.00025,
    "ljp": 14.
}

cell.read_recordings(
    protocol_data=[files_metadata], 
    protocol_name="IDRest"
)


If we wish to extract efeatures while using some settings of eFEL, we can pass a dictionary where the keys are the name of the settings and the values are the value of the settings:

In [12]:
from pprint import pprint

interesting_efeatures = [
    'Spikecount',
    'mean_frequency',
    'ISI_CV',
    'AP1_amp',
    'AP_width'
]

efel_settings = {
    'strict_stiminterval': True, 
    'Threshold': -10.
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=interesting_efeatures,
    efel_settings=efel_settings
)

pprint(cell.recordings[0].efeatures)

{'AP1_amp': 66.47175655411542,
 'AP_width': 1.07777777777705,
 'ISI_CV': 0.4542672780653756,
 'Spikecount': 9.0,
 'mean_frequency': 4.6922655822351}


When using the higher level method bluepyefe.extract.extract_efeatures, the eFEL settings can be passed both as global settings or as settings for a specific feature. 

If both are informed, the feature specific settings will overwrite the global settings. For example, in the following example, the efeature Spikecount will use the global settings but the efeature mean_frequency will use its own settings.

In [13]:
import glob
import bluepyefe.extract

files_metadata = {"B6": {"IDRest": []}, "B8": {"IDRest": []}}

for cell_name in ["B6", "B8"]:
    for file in glob.glob(f"../tests/exp_data/{cell_name}/{cell_name}_Ch0_IDRest_*.ibw"):

        files_metadata[cell_name]["IDRest"].append({
            "i_file": file,
            "v_file": file.replace("Ch0", "Ch3"),
            "i_unit": "pA",
            "v_unit": "mV",
            "t_unit": "s",
            "dt": 0.00025,
            "ljp": 14.
        })

efel_settings = {
    'strict_stiminterval': True, 
    'Threshold': -10.
}

targets = [
    {
        "efeature": "Spikecount",
        "protocol": "IDRest",
        "amplitude": [150],
        "tolerance": [10]
    },
    {
        "efeature": "mean_frequency",
        "protocol": "IDRest",
        "amplitude": [150],
        "tolerance": [10],
        "efel_settings": {
            'strict_stiminterval': False,
            'Threshold': 20.,
        }
    }
]

efeatures, protocols, currents = bluepyefe.extract.extract_efeatures(
    output_directory='MouseCells',
    files_metadata=files_metadata,
    targets=targets,
    threshold_nvalue_save=1,
    protocols_rheobase=[],
    efel_settings=efel_settings
)

Cannot compute the relative current amplitude for the recordings of cell B6 because its rheobase is None.
Cannot compute the relative current amplitude for the recordings of cell B8 because its rheobase is None.
Number of values < threshold_nvalue_save for efeature Spikecount stimulus IDRest_[150]. The efeature will be ignored
Number of values < threshold_nvalue_save for efeature mean_frequency stimulus IDRest_[150]. The efeature will be ignored
No efeatures for stimulus IDRest_[150]. The protocol will not be created.
The output of the extraction is empty. Something went wrong. Please check that your targets, files_metadata and protocols_rheobase match the data you have available.


Note that if both global eFEL settings and feature specific settings are used, the resulting set of settings is the union of both sets of settings. For example, if the global settings are:
```{
    'strict_stiminterval': True, 
    'Threshold': -10.
}```

and the efeature settings are:
```{
    'strict_stiminterval': False,
    'interp_step': 0.1,
}```

The final effective settings that will be used are:
```{
    'strict_stiminterval': False,
    'interp_step': 0.1,
    'Threshold': -10.
}```

## The default eFEL settings and the default BluePyEfe settings

If no settings are passed to the function bluepyefe.extract.extract_efeatures, the default settings that will be used are:
```{
    'strict_stiminterval': True,
    'Threshold': -20.,
    'interp_step': 0.025
}```

Note that these default settings are different from the one used by default by eFEL.

If you do not wish to used default settings, you will have to explicitly pass an empty dictionary as an argument:
```bluepyefe.extract.extract_efeatures(
    output_directory='MouseCells',
    files_metadata=files_metadata,
    targets=targets,
    threshold_nvalue_save=1,
    efel_settings={}
)```

## The Threshold setting

The settings "Threshold", expects a float. This settings set the voltage threshold above which a spike in the voltage trace is detected as an action potential.

In [14]:
efel_settings = {
    'Threshold': -10.
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount'],
    efel_settings=efel_settings
)

print('Number of spikes with Threshold = -10.:  ', cell.recordings[0].efeatures['Spikecount'])

Number of spikes with Threshold = -10.:   9.0


In [15]:
efel_settings = {
    'Threshold': 50.
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount'],
    efel_settings=efel_settings
)

print('Number of spikes with Threshold = 50.:  ', cell.recordings[0].efeatures['Spikecount'])

Number of spikes with Threshold = 50.:   0.0


## The strict_stiminterval, stim_start and stim_end settings

The settings "strict_stiminterval" is a boolean. It specifies if the computation of the efeature should depend on the start and the end of the current stimulus.

When "strict_stiminterval" is set to True, the stim_start and stim_end (float in ms) should also be informed. By default, BluePyEfe set these values to the start and the end of the current stimulus, but you can also pass a value by hand.

In [16]:
efel_settings = {
    'Threshold': -10.,
    'strict_stiminterval': True
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount'],
    efel_settings=efel_settings
)

print('Number of spikes during the step:  ', cell.recordings[0].efeatures['Spikecount'])

Number of spikes during the step:   9.0


In [17]:
efel_settings = {
    'Threshold': -10.,
    'strict_stiminterval': True,
    'stim_start': 0.,
    'stim_end': 100.,
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount'],
    efel_settings=efel_settings
)

print('Number of spikes between t=0ms and t=100ms:  ', cell.recordings[0].efeatures['Spikecount'])

Number of spikes between t=0ms and t=100ms:   0.0


Note that the setting strict_stiminterval affects differently the computation of different features.
For example, for the features ohmic_input_resistance or voltage_deflection, strict_stiminterval should always be True and the stim_start and stim_end should always match the beginning and the end of the current stimulus.
That is because these efeatures depend explicitly on the difference in voltage induced by the stimulus.


## The interp_step setting

The settings "interp_step" is a float, its unit is ms.

When extraction efeatures from a time series, eFEL re-interpolate the voltage based on a fix time step. And the efeature value is computed from the interpolated time series and not from the original one. The settings "interp_step" fixes the time step used during the interpolation.

It could be expected that all efeature do not vary with interp_step, however, the computation of some efeatures such as AP_duration_half_width make use of specific number of data points and will vary with this setting:

In [18]:
efel_settings = {
    'Threshold': -10.,
    'strict_stiminterval': True,
    'interp_step': 0.1
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount', 'AP_width'],
    efel_settings=efel_settings
)

print('Number of spikes at interp_step=0.1ms:  ', cell.recordings[0].efeatures['Spikecount'])
print('Mean width of spikes at interp_step=0.1ms:  ', cell.recordings[0].efeatures['AP_width'])

Number of spikes at interp_step=0.1ms:   9.0
Mean width of spikes at interp_step=0.1ms:   1.07777777777705


In [19]:
efel_settings = {
    'Threshold': -10.,
    'strict_stiminterval': True,
    'interp_step': 0.025
}

cell.extract_efeatures(
    protocol_name='IDRest', 
    efeatures=['Spikecount', 'AP_width'],
    efel_settings=efel_settings
)

print('Number of spikes at interp_step=0.025ms:  ', cell.recordings[0].efeatures['Spikecount'])
print('Mean width of spikes at interp_step=0.025ms:  ', cell.recordings[0].efeatures['AP_width'])

Number of spikes at interp_step=0.025ms:   9.0
Mean width of spikes at interp_step=0.025ms:   1.0805555555584507


In [20]:
assert cell.recordings[0].efeatures['Spikecount'] == 9
assert cell.recordings[0].efeatures['AP_width'] == 1.0805555555584507

Note that the default "inter_step" in BluePyEfe is 0.025ms, while it is 0.1ms in eFEL. Overall, it is advised not to go above 0.1ms.